## Create dimension/fact tables

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.functions import substring
from pyspark.sql.functions import col, unix_timestamp, months_between, round
from pyspark.sql.functions import year, quarter, month, weekofyear, dayofmonth

In [0]:
# read in stations data
stations = spark.read.format("delta") \
    .load("/delta/stations")

# Rename columns
columns = ['station_id', 'name', 'latitude', 'longitude']
dim_stations = stations.toDF(*columns) 
dim_stations.show(5) 

# Write dim_stations to table
dim_stations.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_stations")

+------------+--------------------+-----------------+------------------+
 station_id| name| latitude| longitude|
+------------+--------------------+-----------------+------------------+
 525|Glenwood Ave & To...| 42.012701|-87.66605799999999|
KA1503000012| Clark St & Lake St|41.88579466666667|-87.63110066666668|
 637|Wood St & Chicago...| 41.895634| -87.672069|
 13216| State St & 33rd St| 41.8347335| -87.6258275|
 18003|Fairbanks St & Su...|41.89580766666667|-87.62025316666669|
+------------+--------------------+-----------------+------------------+
only showing top 5 rows

In [0]:
# read in riders data
riders = spark.read.format("delta") \
    .load("/delta/riders")

# Rename columns
columns = ['rider_id', 'first_name', 'last_name', 'address', 'birthday', 'account_start_date', 'account_end_date', 'is_member']
dim_riders = riders.toDF(*columns) 
dim_riders.show(5) 

# Write dim_riders to table
dim_riders.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_riders")

+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
rider_id|first_name|last_name| address| birthday| account_start_date| account_end_date|is_member|
+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
 1000| Diana| Clark| 1200 Alyssa Squares|1989-02-13 00:00:00|2019-04-23 00:00:00| null| true|
 1001| Jennifer| Smith| 397 Diana Ferry|1976-08-10 00:00:00|2019-11-01 00:00:00|2020-09-01 00:00:00| true|
 1002| Karen| Smith|644 Brittany Row ...|1998-08-10 00:00:00|2022-02-04 00:00:00| null| true|
 1003| Bryan| Roberts|996 Dickerson Tur...|1999-03-29 00:00:00|2019-08-26 00:00:00| null| false|
 1004| Jesse|Middleton|7009 Nathan Expre...|1969-04-11 00:00:00|2019-09-14 00:00:00| null| true|
+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
only showing top 5 rows

In [0]:
# read in payment data
payments = spark.read.format("delta") \
    .load("/delta/payments")

# Rename columns
columns = ['payment_id', 'date', 'amount', 'rider_id']
payments = payments.toDF(*columns) 

# Convert the "date" column to integer type
fact_payments = payments.withColumn("date", payments["date"].cast("string"))  # Ensure it's a string
fact_payments = fact_payments.withColumn("date", regexp_replace(fact_payments["date"], "-", ""))  # Remove '-'
fact_payments = fact_payments.withColumn("date", substring(fact_payments["date"],1,8).cast("long")) 

# rename date to date_id
fact_payments = fact_payments.withColumnRenamed("date", "date_id")
fact_payments = fact_payments.select("payment_id","amount","rider_id","date_id")  # rearrange columns

# Show table
fact_payments.show(5)

# Write fact_payment to table
fact_payments.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_payment")

+----------+------+--------+--------+
payment_id|amount|rider_id| date_id|
+----------+------+--------+--------+
 1| 9.0| 1000|20190501|
 2| 9.0| 1000|20190601|
 3| 9.0| 1000|20190701|
 4| 9.0| 1000|20190801|
 5| 9.0| 1000|20190901|
+----------+------+--------+--------+
only showing top 5 rows

In [0]:
# read in trips data
trips = spark.read.format("delta") \
    .load("/delta/trips")

# Rename columns
columns = ['trip_id', 'rideable_type', 'start_at', 'end_at', 'start_station_id', 'end_station_id', 'rider_id']
trips = trips.toDF(*columns) 

# create age column
trips = trips.join(dim_riders[["rider_id", "birthday"]], on = "rider_id", how = "left")
trips = trips.withColumn("age", (months_between(trips["start_at"], trips["birthday"]) / 12))
trips = trips.withColumn("age", round(trips["age"], 0))

# Create duration in minutess column
trips = trips.withColumn("duration", (unix_timestamp("end_at") - unix_timestamp("start_at"))/60)
trips = trips.withColumn("duration", round(trips["duration"], 3))

# Create date_id column
trips = trips.withColumn("date_id", trips["start_at"].cast("string"))  # Ensure it's a string
trips = trips.withColumn("date_id", regexp_replace(trips["date_id"], "-", ""))  # Remove '-'
trips = trips.withColumn("date_id", substring(trips["date_id"],1,8).cast("long"))

# Rearrange columns
fact_trips = trips.select("trip_id","rideable_type","start_at","end_at", "duration", "age", "start_station_id", "end_station_id", "rider_id", "date_id")  
fact_trips.show(5)

# Write fact_trips to table
fact_trips.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_trips")

+----------------+-------------+-------------------+-------------------+--------+----+----------------+--------------+--------+--------+
 trip_id|rideable_type| start_at| end_at|duration| age|start_station_id|end_station_id|rider_id| date_id|
+----------------+-------------+-------------------+-------------------+--------+----+----------------+--------------+--------+--------+
89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43| 6.783|37.0| 525| 660| 71934|20210212|
0FEFDE2603568365| classic_bike|2021-02-14 17:52:38|2021-02-14 18:12:09| 19.517|38.0| 525| 16806| 47854|20210214|
E6159D746B2DBB91|electric_bike|2021-02-09 19:10:18|2021-02-09 19:19:10| 8.867|33.0| KA1503000012| TA1305000029| 70870|20210209|
B32D3199F1C2E75B| classic_bike|2021-02-02 17:49:41|2021-02-02 17:54:06| 4.417|19.0| 637| TA1305000034| 58974|20210202|
83E463F23575F4BF|electric_bike|2021-02-23 15:07:23|2021-02-23 15:22:37| 15.233|72.0| 13216| TA1309000055| 39608|20210223|
+----------------+-------------+-------------------+-------------------+--------+----+----------------+--------------+--------+--------+
only showing top 5 rows

In [0]:
# Create date dimension
date_df = fact_trips.select('start_at').union(payments.select('date'))
date_df = date_df.withColumn("start_at", col("start_at").cast("date"))
date_df = date_df.dropDuplicates(["start_at"])
date_df = date_df.withColumnRenamed("start_at", "date")

# Extract year, quarter, month, week, and day of the month into separate columns
date_df = date_df.withColumn("year", year(date_df["date"]))
date_df = date_df.withColumn("quarter", quarter(date_df["date"]))
date_df = date_df.withColumn("month", month(date_df["date"]))
date_df = date_df.withColumn("week", weekofyear(date_df["date"]))
date_df = date_df.withColumn("day", dayofmonth(date_df["date"]))

# Create date_id column
date_df = date_df.withColumn("date_id", date_df["date"].cast("string"))  # Ensure it's a string
date_df = date_df.withColumn("date_id", regexp_replace(date_df["date_id"], "-", ""))  # Remove '-'
date_df = date_df.withColumn("date_id", substring(date_df["date_id"],1,8).cast("long"))

# Rearrange columns
dim_date = date_df.select("date_id","date","year","quarter", "month", "week", "day")  
dim_date.show(5)

# Write fact_trips to table
dim_date.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_date")

+--------+----------+----+-------+-----+----+---+
 date_id| date|year|quarter|month|week|day|
+--------+----------+----+-------+-----+----+---+
20210622|2021-06-22|2021| 2| 6| 25| 22|
20210604|2021-06-04|2021| 2| 6| 22| 4|
20210627|2021-06-27|2021| 2| 6| 25| 27|
20210620|2021-06-20|2021| 2| 6| 24| 20|
20210628|2021-06-28|2021| 2| 6| 26| 28|
+--------+----------+----+-------+-----+----+---+
only showing top 5 rows